In [2]:
import numpy as np
import pandas as pd

1. on指定以哪个键进行连接，以多个列为键合并时，on指定列表
2. how 参数来代表连接形式，分为左连接 left 、右连接 right 、内连接 inner 、外连接 outer
3. 如果两个表中想要连接的列不具备相同的列名，可以通过 left_on 和 right_on 指定
4. 如果两个表中的列出现了重复的列名，那么可以通过 suffixes 参数指定
5. validate 参数来检查连接的唯一性模式，这里共有三种模式，即一对一连接 1:1 ，一对多连接 1:m ，多对一连接 m:1 连接，第一个是指左右表的键都是唯一的，后面两个分别指左表键唯一和右表键唯一，指定的模式与表格不匹配则会报错

## merge—值连接

In [3]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'], 'Gender':['F','M']})
df1.merge(df2, on='Name', how='left') #左连接

,Name,Age,Gender
0,San Zhang,20,NaN
1,Si Li,30,F


In [6]:
df1.merge(df2, on='Name', how='right') #右连接
df1.merge(df2, on='Name', how='inner') #内连接
df1.merge(df2, on='Name', how='outer') #右连接

,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,F
2,Wu Wang,NaN,M


In [8]:
# 如果两个表中想要连接的列不具备相同的列名，可以通过 left_on 和 right_on 指定
df1 = pd.DataFrame({'df1-Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'df2-Name':['Si Li','Wu Wang'], 'Gender':['F','M']})
df1.merge(df2, left_on='df1-Name', right_on='df2-Name',how='outer') 

,df1-Name,Age,df2-Name,Gender
0,San Zhang,20.0,NaN,NaN
1,Si Li,30.0,Si Li,F
2,NaN,NaN,Wu Wang,M


In [14]:
# 如果两个表中的列出现了重复的列名，那么可以通过 suffixes 参数指定
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':['80']})
df2 = pd.DataFrame({'Name':['Si Li'],'Grade':['90']})
df1.merge(df2, on='Name', suffixes=['shuxue','yingyu'], how='outer')

,Name,Gradeshuxue,Gradeyingyu
0,San Zhang,80,NaN
1,Si Li,NaN,90


In [15]:
# 以多个列为键合并时，on指定列表
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],'Age':[20, 21],'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],'Gender':['F', 'M'],'Class':['two', 'one']})
df1.merge(df2, on=['Name','Class']) # how默认为inner

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


In [29]:
#validate 参数来检查连接的唯一性模式
#这里共有三种模式，即一对一连接 1:1 ，一对多连接 1:m ，多对一连接 m:1 连接
#第一个是指左右表的键都是唯一的，后面两个分别指左表键唯一和右表键唯一
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li'],'Age':[20, 21],'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],'Gender':['F', 'M'],'Class':['one', 'one']})
df1.merge(df2,on=['Name','Class'],validate='1:m',how='outer')  #validate='1:1' 或者 'm:1'会报错

,Name,Age,Class,Gender
0,San Zhang,20,one,F
1,San Zhang,20,one,M
2,Si Li,21,two,NaN


## join—索引连接

- join 函数来处理索引连接，它的参数选择要少于 merge ，除了必须的 on 和 how 之外，可以对重复的列指定左右后缀 lsuffix 和 rsuffix 
- on 参数指索引名，单层索引时省略参数表示按照当前索引连接

In [36]:
#df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]},)
#df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'], 'Gender':['F','M']})
df1 = pd.DataFrame({'Age':[20,30]},index=pd.Series(['San Zhang','Si Li'],name="Name"))
df2 = pd.DataFrame({'Gender':['F','M']},index=pd.Series(['Si Li','Wu Wang'],name='Name'))
df1.join(df2,how='outer') #how默认为left

,Age,Gender
Name,,
San Zhang,20.0,NaN
Si Li,30.0,F
Wu Wang,NaN,M


In [39]:
# 重复列时指定后缀
df1 = pd.DataFrame({'Grade':[20,30]},index=pd.Series(['San Zhang','Si Li'],name="Name"))
df2 = pd.DataFrame({'Grade':[70,80]},index=pd.Series(['Si Li','Wu Wang'],name='Name'))
df1.join(df2, lsuffix='df1', rsuffix='df2',how='outer')

,Gradedf1,Gradedf2
Name,,
San Zhang,20.0,NaN
Si Li,30.0,70.0
Wu Wang,NaN,80.0


In [45]:
# 以多列为键时，join使用多级索引
df1 = pd.DataFrame({'Age':[20,21]},index=pd.MultiIndex.from_arrays([['zhangsan','zhangsan'],['one','two']],names=['name','class']))
df2 = pd.DataFrame({'Gender':['F','M']},index=pd.MultiIndex.from_arrays([['zhangsan','zhangsan'],['two','one']],names=['name','class']))
df1.join(df2)

Age Gender
name     class            
zhangsan one     20      M
         two     21      F

## concat-方向连接

两个或多个表纵向或横向连接
- axis:拼接方向,0纵向，1横向，默认为0
- join:连接形式，inner,outer,left,right
- keys: 指定数据来源于哪个表

In [51]:
## 纵向合并record,纵向拼接会根据列索引对其，默认状态下 join=outer ，表示保留所有的列，并将不存在的值设为缺失
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Gender':['M']})
pd.concat([df1, df2], ignore_index=True)

,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,NaN
2,Wu Wang,NaN,M


In [50]:
## 横向合并字段
df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})
pd.concat([df1,df2,df3],axis=1)

,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


In [54]:
#  keys 参数的使用场景在于多个表合并后，用户仍然想要知道新表中的数据来自于哪个原表，这时可以通过 keys 参数产生多级索引进行标记
pd.concat([df1, df2], keys=['df1', 'df2'])

Name   Age Gender
df1 0  San Zhang  20.0    NaN
    1      Si Li  30.0    NaN
df2 0    Wu Wang   NaN      M

## append/assign-序列与表的合并

* append：将一个序列追加到表的行末
* assign：将一个序列追加到表的列末

In [58]:
# 如果原表是默认整数序列的索引，那么可以使用 ignore_index=True 对新序列对应的索引自动标号，否则必须对 Series 指定 name 属性
s = pd.Series(['Wu Wang', 21], index = df1.columns)
df1.append(s,ignore_index=True)

,Name,Age
0,San Zhang,20
1,Si Li,30
2,Wu Wang,21


In [61]:
#一般通过 df['new_col'] = ... 的形式就可以等价地添加新列。
#区别在于使用[]会直接在原表上进行改动，而 assign 返回的是一个临时副本。
s2 = pd.Series([80, 90])
df1.assign(Grade=s2)

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,30,90


## compare/combine

* compare比较两个表的不同之处，并将其汇总展示(需要传入功能函数)
    - 结果中返回了不同值所在的行列，如果相同则会被填充为缺失值 NaN ，其中 other 和 self 分别指代传入的参数表和被调用的表自身。
    - 如果想要完整显示表中所有元素的比较情况，可以设置 keep_shape=True 
* combine能够让两张表按照一定的规则进行组合，在进行规则比较时会自动进行列索引的对齐。
    - overtwrite 参数为 False 可以保留 被调用表 中未出现在传入的参数表中的列，而不会设置未缺失值
* combine_first在对两张表组合时，若第二张表中的值在第一张表中对应索引位置的值不是缺失状态，那么就使用第一张表的值填充

In [62]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],'Age':[20, 21 ,21],'Class':['one', 'two', 'three']})
df2 = pd.DataFrame({'Name':['San Zhang', 'si Li', 'Wu Wang'],'Age':[20, 21 ,21],'Class':['One', 'two', 'three']})
df1.compare(df2)

Name        Class      
    self  other  self other
0    NaN    NaN   one   One
1  Si Li  si Li   NaN   NaN

In [63]:
df1.compare(df2,keep_shape=True)

Name         Age       Class      
    self  other self other  self other
0    NaN    NaN  NaN   NaN   one   One
1  Si Li  si Li  NaN   NaN   NaN   NaN
2    NaN    NaN  NaN   NaN   NaN   NaN

In [65]:
# 选出对应索引位置较小的元素
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1) # 两个索引的并集
    res = s1.where(s1<s2, s2)
    res = res.mask(s1.isna()) # isna表示是否为缺失值，返回布尔序列
    return res
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])

In [66]:
df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


In [67]:
df1.combine(df2, choose_min, overwrite=False)

,A,B,C,D
0,1.0,NaN,NaN,NaN
1,2.0,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


In [76]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1) # 两个索引的并集
    res = s1.where(s1<s2, s2) 
    res = res.mask(s1.isna(),s2) # isna表示是否为缺失值，返回布尔序列
    res = res.mask(s2.isna(),s1) # s2为空时，返回s1
#    res = res.where(s1.isna(),s1) #s1不为空时，返回s1
    return res
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,10]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])
df1.combine(df2, choose_min,overwrite=False)

,A,B,C,D
0,1.0,3.0,5.0,NaN
1,2.0,4.0,7.0,9.0
2,NaN,6.0,8.0,10.0


In [71]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,np.nan]})
df2 = pd.DataFrame({'A':[5,6], 'B':[7,8]}, index=[1,2])
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0
